# Análise de Dados Financeiros da Amazon

Neste notebook, vamos analisar dados financeiros históricos da Amazon usando a arquitetura Medallion do DataFlow Lab. Os dados serão processados através das camadas Bronze, Silver e Gold, e então utilizados para análises e modelagem preditiva.

## Configuração do Ambiente

Primeiro, vamos configurar o ambiente Spark com suporte ao Delta Lake e conexão ao MinIO.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum, count, window, lag, date_format
from pyspark.sql.types import DoubleType, DateType
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import pandas as pd

# Configurar SparkSession com Delta Lake e MinIO
spark = (
    SparkSession.builder.appName("AmazonFinancialAnalysis")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

## Ingestão de Dados (Camada Bronze)

Vamos ingerir dados históricos de ações da Amazon e salvá-los na camada Bronze sem transformações significativas.

In [ ]:
# Baixar dados históricos da Amazon (exemplo)
# Em um caso real, esses dados seriam obtidos de uma API financeira como Yahoo Finance, Alpha Vantage, etc.
# Para este exemplo, vamos simular criando um DataFrame

# Criar dados simulados para demonstração
data = [
    ("2025-01-02", 3400.50, 3450.00, 3380.00, 3440.25, 2500000),
    ("2025-01-03", 3440.25, 3500.00, 3430.00, 3490.75, 3000000),
    ("2025-01-04", 3490.75, 3520.00, 3470.00, 3510.50, 2800000),
    ("2025-01-05", 3510.50, 3550.00, 3490.00, 3540.00, 3200000),
    ("2025-01-06", 3540.00, 3580.00, 3520.00, 3570.25, 2900000),
    ("2025-01-09", 3570.25, 3600.00, 3550.00, 3590.50, 2600000),
    ("2025-01-10", 3590.50, 3620.00, 3570.00, 3610.75, 2700000),
    ("2025-01-11", 3610.75, 3650.00, 3600.00, 3640.00, 3100000),
    ("2025-01-12", 3640.00, 3670.00, 3620.00, 3660.50, 2800000),
    ("2025-01-13", 3660.50, 3700.00, 3640.00, 3685.75, 3300000),
    ("2025-01-16", 3685.75, 3710.00, 3660.00, 3695.00, 2500000),
    ("2025-01-17", 3695.00, 3730.00, 3680.00, 3720.25, 2900000),
    ("2025-01-18", 3720.25, 3750.00, 3700.00, 3740.50, 3000000),
    ("2025-01-19", 3740.50, 3770.00, 3720.00, 3760.75, 2800000),
    ("2025-01-20", 3760.75, 3800.00, 3740.00, 3780.00, 3200000),
    ("2025-01-23", 3780.00, 3810.00, 3760.00, 3790.25, 2700000),
    ("2025-01-24", 3790.25, 3820.00, 3770.00, 3810.50, 2900000),
    ("2025-01-25", 3810.50, 3840.00, 3790.00, 3830.75, 3100000),
    ("2025-01-26", 3830.75, 3860.00, 3810.00, 3850.00, 3000000),
    ("2025-01-27", 3850.00, 3880.00, 3830.00, 3870.25, 3200000),
    ("2025-01-30", 3870.25, 3900.00, 3850.00, 3890.50, 2800000),
    ("2025-01-31", 3890.50, 3920.00, 3870.00, 3910.75, 3000000),
    ("2025-02-01", 3910.75, 3940.00, 3890.00, 3930.00, 3100000),
    ("2025-02-02", 3930.00, 3950.00, 3900.00, 3940.25, 2700000),
    ("2025-02-03", 3940.25, 3970.00, 3920.00, 3960.50, 2900000),
    ("2025-02-06", 3960.50, 3990.00, 3940.00, 3980.75, 3000000),
    ("2025-02-07", 3980.75, 4010.00, 3960.00, 4000.00, 3300000),
    ("2025-02-08", 4000.00, 4030.00, 3980.00, 4020.25, 3500000),
    ("2025-02-09", 4020.25, 4050.00, 4000.00, 4040.50, 3200000),
    ("2025-02-10", 4040.50, 4070.00, 4020.00, 4060.75, 3100000),
]

# Criar DataFrame
schema = ["date", "open", "high", "low", "close", "volume"]
raw_df = spark.createDataFrame(data, schema=schema)

# Converter coluna de data para tipo correto
raw_df = raw_df.withColumn("date", col("date").cast(DateType()))

In [ ]:
# Adicionar metadados para a camada Bronze
from pyspark.sql.functions import current_timestamp, lit

bronze_df = (
    raw_df.withColumn("ingestion_timestamp", current_timestamp())
    .withColumn("source", lit("yahoo_finance"))
    .withColumn("symbol", lit("AMZN"))
)

# Salvar na camada Bronze
bronze_df.write.format("delta").mode("overwrite").save("s3a://bronze/stock_data/amazon")

## Processamento para a Camada Silver

Vamos processar os dados da camada Bronze para a camada Silver, realizando limpeza e transformações necessárias.

In [ ]:
# Ler dados da camada Bronze
bronze_data = spark.read.format("delta").load("s3a://bronze/stock_data/amazon")

# Processamento para a camada Silver
# - Converter tipos de dados
# - Remover dados inválidos
# - Adicionar colunas derivadas

silver_df = (
    bronze_data.withColumn("open", col("open").cast(DoubleType()))
    .withColumn("high", col("high").cast(DoubleType()))
    .withColumn("low", col("low").cast(DoubleType()))
    .withColumn("close", col("close").cast(DoubleType()))
    .withColumn("volume", col("volume").cast(DoubleType()))
    .withColumn("day_of_week", date_format(col("date"), "E"))
    .withColumn("month", date_format(col("date"), "MM"))
)

# Calcular colunas derivadas
silver_df = (
    silver_df.withColumn(
        "daily_return", (col("close") - col("open")) / col("open") * 100
    )
    .withColumn("range", col("high") - col("low"))
    .withColumn("is_up_day", col("close") > col("open"))
)

# Filtrar dados inválidos
silver_df = (
    silver_df.filter(col("open") > 0)
    .filter(col("high") > col("low"))
    .filter(col("volume") > 0)
)

# Salvar na camada Silver
silver_df.write.format("delta").mode("overwrite").save("s3a://silver/stock_data/amazon")

## Análise e Agregação para a Camada Gold

Vamos processar os dados da camada Silver para a camada Gold, realizando agregações e criando métricas analíticas.

In [ ]:
# Ler dados da camada Silver
silver_data = spark.read.format("delta").load("s3a://silver/stock_data/amazon")

# Calcular métricas diárias
daily_metrics = silver_data.select(
    "date",
    "symbol",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "daily_return",
    "range",
    "is_up_day",
)

# Calcular métricas móveis (médias móveis de 5 dias)
window_spec = Window.partitionBy("symbol").orderBy("date").rowsBetween(-4, 0)

moving_metrics = daily_metrics.withColumn(
    "ma5_close", avg(col("close")).over(window_spec)
).withColumn("ma5_volume", avg(col("volume")).over(window_spec))

# Calcular variação diária
window_lag = Window.partitionBy("symbol").orderBy("date")
gold_df = moving_metrics.withColumn(
    "prev_close", lag(col("close"), 1).over(window_lag)
).withColumn(
    "daily_change_pct", (col("close") - col("prev_close")) / col("prev_close") * 100
)

# Filtrar valores nulos (primeiro dia não terá variação calculável)
gold_df = gold_df.filter(col("daily_change_pct").isNotNull())

# Salvar na camada Gold
gold_df.write.format("delta").mode("overwrite").save(
    "s3a://gold/stock_data/amazon_metrics"
)

## Análise Exploratória

Vamos usar os dados da camada Gold para realizar algumas análises exploratórias.

In [ ]:
# Carregar dados da camada Gold
gold_data = spark.read.format("delta").load("s3a://gold/stock_data/amazon_metrics")

# Converter para Pandas para visualização
pd_df = gold_data.toPandas()

In [ ]:
# Visualização do preço de fechamento e média móvel de 5 dias
plt.figure(figsize=(12, 6))
plt.plot(pd_df["date"], pd_df["close"], label="Preço de Fechamento")
plt.plot(
    pd_df["date"], pd_df["ma5_close"], label="Média Móvel de 5 dias", linestyle="--"
)
plt.title("Preço da Ação da Amazon - Janeiro/Fevereiro 2025")
plt.xlabel("Data")
plt.ylabel("Preço ($)")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Análise da variação percentual diária
plt.figure(figsize=(12, 6))
plt.bar(
    pd_df["date"],
    pd_df["daily_change_pct"],
    color=["g" if x > 0 else "r" for x in pd_df["daily_change_pct"]],
)
plt.title("Variação Percentual Diária da Ação da Amazon")
plt.xlabel("Data")
plt.ylabel("Variação (%)")
plt.axhline(y=0, color="black", linestyle="-", alpha=0.3)
plt.grid(axis="y", alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Volume de negociação e média móvel de volume
plt.figure(figsize=(12, 6))
plt.bar(pd_df["date"], pd_df["volume"], alpha=0.5, label="Volume")
plt.plot(
    pd_df["date"],
    pd_df["ma5_volume"],
    color="orange",
    label="Média Móvel Volume (5 dias)",
)
plt.title("Volume de Negociação da Amazon")
plt.xlabel("Data")
plt.ylabel("Volume")
plt.legend()
plt.grid(axis="y", alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Estatísticas e Insights

Vamos extrair algumas estatísticas e insights dos dados.

In [ ]:
# Estatísticas descritivas
print("Estatísticas dos Preços de Fechamento:")
pd_df["close"].describe()

In [ ]:
# Análise de dias positivos vs. negativos
dias_positivos = pd_df["daily_change_pct"].apply(lambda x: x > 0).sum()
dias_negativos = pd_df["daily_change_pct"].apply(lambda x: x <= 0).sum()
total_dias = len(pd_df)

print(
    f"Dias com variação positiva: {dias_positivos} ({dias_positivos/total_dias*100:.2f}%)"
)
print(
    f"Dias com variação negativa: {dias_negativos} ({dias_negativos/total_dias*100:.2f}%)"
)

# Calcular retorno médio diário
retorno_medio = pd_df["daily_change_pct"].mean()
print(f"Retorno médio diário: {retorno_medio:.4f}%")

# Calcular volatilidade (desvio padrão do retorno diário)
volatilidade = pd_df["daily_change_pct"].std()
print(f"Volatilidade diária: {volatilidade:.4f}%")

## Modelagem Preditiva

Vamos criar um modelo simples para prever o preço de fechamento usando regressão linear.

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import mlflow

# Configurar MLflow
mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.set_experiment("amazon_stock_prediction")

# Preparar features
feature_cols = ["open", "high", "low", "volume", "ma5_close", "ma5_volume"]
label_col = "close"

# Criar assembler para combinar features
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
assembled_data = assembler.transform(gold_data)

In [ ]:
# Dividir em conjuntos de treino e teste (70% treino, 30% teste)
train_data, test_data = assembled_data.randomSplit([0.7, 0.3], seed=42)

# Treinar modelo de regressão linear
with mlflow.start_run(run_name="linear_regression_model"):
    # Criar e treinar o modelo
    lr = LinearRegression(featuresCol="features", labelCol=label_col)
    lr_model = lr.fit(train_data)

    # Fazer previsões
    predictions = lr_model.transform(test_data)

    # Avaliar o modelo
    evaluator = RegressionEvaluator(labelCol=label_col, predictionCol="prediction")
    rmse = evaluator.setMetricName("rmse").evaluate(predictions)
    r2 = evaluator.setMetricName("r2").evaluate(predictions)

    # Registrar parâmetros e métricas no MLflow
    mlflow.log_params(
        {
            "features": ", ".join(feature_cols),
            "elasticNetParam": lr.getElasticNetParam(),
            "regParam": lr.getRegParam(),
        }
    )

    mlflow.log_metrics({"rmse": rmse, "r2": r2})

    # Salvar modelo
    mlflow.spark.log_model(lr_model, "model")

    # Registrar coeficientes
    coefficients = {
        feature: coef for feature, coef in zip(feature_cols, lr_model.coefficients)
    }
    mlflow.log_params({f"coef_{k}": v for k, v in coefficients.items()})
    mlflow.log_param("intercept", lr_model.intercept)

    # Imprimir resultados
    print(f"RMSE: {rmse}")
    print(f"R²: {r2}")
    print(f"Coeficientes: {coefficients}")
    print(f"Intercepto: {lr_model.intercept}")

In [ ]:
# Visualizar previsões vs. valores reais
pred_df = predictions.select("date", label_col, "prediction").toPandas()

plt.figure(figsize=(12, 6))
plt.plot(pred_df["date"], pred_df[label_col], "o-", label="Valor Real")
plt.plot(pred_df["date"], pred_df["prediction"], "o--", label="Previsão")
plt.title("Previsão do Preço de Fechamento da Ação da Amazon")
plt.xlabel("Data")
plt.ylabel("Preço ($)")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Conclusão

Neste notebook, demonstramos o fluxo completo de dados através da arquitetura Medallion no DataFlow Lab:

1. **Camada Bronze**: Ingestão de dados brutos da ação da Amazon
2. **Camada Silver**: Transformação, limpeza e enriquecimento dos dados
3. **Camada Gold**: Agregação e criação de métricas analíticas

Além disso, realizamos:
- Análise exploratória dos dados com visualizações
- Extração de insights estatísticos
- Modelagem preditiva de preços com MLflow para rastreamento

Este fluxo demonstra a eficácia da arquitetura Medallion do DataFlow Lab para análises financeiras, combinando a confiabilidade do Delta Lake com o poder analítico do Spark e a rastreabilidade do MLflow.